In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, InputLayer, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [15]:
# Image specifications
img_height = 128
img_width = 128
batch_size = 32

In [16]:
# Prepare ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [18]:
# Define the model architecture
model = Sequential([
    InputLayer(input_shape=(img_height, img_width, 3)),  # 3 channels for RGB images
    Conv2D(32, (3, 3), activation='relu'),  # First convolution layer
    MaxPooling2D((2, 2)),  # First max pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Second convolution layer
    MaxPooling2D((2, 2)),  # Second max pooling layer
    Conv2D(128, (3, 3), activation='relu'),  # Third convolution layer
    MaxPooling2D((2, 2)),  # Third max pooling layer
    Flatten(),  # Flatten the output to feed into a DNN
    Dense(128, activation='relu'),  # Dense layer after convolution layers
    Dropout(0.5),  # Dropout layer to reduce overfitting
    Dense(1, activation='sigmoid')  # Output layer with one neuron using sigmoid activation
])

In [19]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [20]:
# Model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
number_of_epochs = 50

In [22]:
# Train the model
history = model.fit(
    train_generator,
    epochs = number_of_epochs,
    validation_data=validation_generator
)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 434ms/step - accuracy: 0.5125 - loss: 1.0482 - val_accuracy: 0.4286 - val_loss: 0.7129
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 264ms/step - accuracy: 0.5631 - loss: 0.6854 - val_accuracy: 0.4286 - val_loss: 0.7251
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 251ms/step - accuracy: 0.5659 - loss: 0.6832 - val_accuracy: 0.4286 - val_loss: 0.7216
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 249ms/step - accuracy: 0.5716 - loss: 0.6861 - val_accuracy: 0.4286 - val_loss: 0.7854
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 252ms/step - accuracy: 0.6157 - loss: 0.6567 - val_accuracy: 0.6071 - val_loss: 0.6901
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 265ms/step - accuracy: 0.5728 - loss: 0.6664 - val_accuracy: 0.4286 - val_loss: 0.7369
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 254ms/step - accuracy: 0.6022 - loss: 0.6602 - val_accuracy: 0.4286 - val_loss: 0.7534
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 251ms/step - accuracy: 0.6026 - loss: 0.6539 - val_accuracy: 0.4643 - val_loss:

In [23]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4643 - loss: 5.5606
Validation accuracy: 0.4642857015132904


In [25]:
# Save the model, optional
model.save(f'Pth_Files/CNN_model_{number_of_epochs}.keras')